**IA & Big Data**

Prof. Miguel Bozer da Silva - miguel.bozer@senaisp.edu.br

---

In [1]:
# Importando as bibliotecas para os modelos
import pandas as pd
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

As tarefas 1 e 2 já foram apresentadas no arquivo 2. Label Encoding e One Hot Encoding.ipynb. Execute-os para podermos trabalhar com a divisão do conjunto de dados.

## Tarefa #1: Recebendo os dados

In [4]:
# Recebendo os dados

carros = pd.read_csv('../data/Used_fiat_500_in_Italy_dataset.csv', sep = ',')
carros.head()

,model,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price
0,pop,69,manual,4474,56779,2,45.071079,7.46403,4490
1,lounge,69,manual,2708,160000,1,45.069679,7.70492,4500
2,lounge,69,automatic,3470,170000,2,45.514599,9.28434,4500
3,sport,69,manual,3288,132000,2,41.903221,12.49565,4700
4,sport,69,manual,3712,124490,2,45.532661,9.03892,4790


## Tarefa #2: Corrigindo os dados

In [5]:
carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   model            380 non-null    object 
 1   engine_power     380 non-null    int64  
 2   transmission     380 non-null    object 
 3   age_in_days      380 non-null    int64  
 4   km               380 non-null    int64  
 5   previous_owners  380 non-null    int64  
 6   lat              380 non-null    float64
 7   lon              380 non-null    float64
 8   price            380 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 26.8+ KB


In [ ]:
carros.shape

(380, 9)

Vamos explorar as colunas que são do tipo `object` para aplicarmos o *One Hot Encoding* ou o *Label Encoding*:

In [ ]:
carros['model'].unique()

array(['pop', 'lounge', 'sport', 'star'], dtype=object)

In [ ]:
carros['transmission'].unique()

array(['manual', 'automatic'], dtype=object)

A coluna model e transmission possuem textos e precisamos corrigir isso

Vamos agora transformar a coluna de transmissão que é uma coluna que possui apenas dois valores possíveis. Para isso, vamos usando o comando o `map`. Se o carro for manual o valor será substituído por 0 e se o carro for automático o valor será substituído por 1:

In [ ]:
carros['transmission'] = carros['transmission'].map({'manual': 0, 'automatic': 1})

In [ ]:
carros['transmission'].unique()

array([0, 1])

Vamos aplicar o One Hot Enconding na coluna *model* para transformar os textos em colunas:

In [ ]:
# Aplicando o One Hot Enconding
modelos = pd.get_dummies(carros["model"], prefix = "modelo")

In [ ]:
modelos.head()

,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,0,1,0,0
1,1,0,0,0
2,1,0,0,0
3,0,0,1,0
4,0,0,1,0


Criamos dessa forma 4 colunas novas que são binárias indicando o modelo do veículo. Vamos agora criar um novo `DataFrame` unindo os `DataFrames` carros e transmissao:

In [ ]:
# Concatenando os dados:
carros_corrigidos = pd.concat([carros, modelos], axis=1)

In [ ]:
# Exibindo o novo Dataframe
carros_corrigidos.head()

,model,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,pop,69,0,4474,56779,2,45.071079,7.46403,4490,0,1,0,0
1,lounge,69,0,2708,160000,1,45.069679,7.70492,4500,1,0,0,0
2,lounge,69,1,3470,170000,2,45.514599,9.28434,4500,1,0,0,0
3,sport,69,0,3288,132000,2,41.903221,12.49565,4700,0,0,1,0
4,sport,69,0,3712,124490,2,45.532661,9.03892,4790,0,0,1,0


Pensando em um modelo de *Machine Learning*, a coluna *model* pode ser excluída, pois ela não seria usada para treinar o modelo.

In [ ]:
carros_corrigidos.drop(columns=['model'], inplace = True)

In [ ]:
carros_corrigidos.head()

,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,69,0,4474,56779,2,45.071079,7.46403,4490,0,1,0,0
1,69,0,2708,160000,1,45.069679,7.70492,4500,1,0,0,0
2,69,1,3470,170000,2,45.514599,9.28434,4500,1,0,0,0
3,69,0,3288,132000,2,41.903221,12.49565,4700,0,0,1,0
4,69,0,3712,124490,2,45.532661,9.03892,4790,0,0,1,0


## Tarefa #3: Dividindo o conjunto de dados

Vamos fazer a divisão do nosso conjunto de dados usando o comando `train_test_split`. Esse comando necessita dos seguintes argumentos:
* Dados de entrada;
* Dados de saída;
* test_size: Tamanho do conjunto de teste em percentual (de 0 até 1)
* random_state: opcional - Forma com a qual as linhas do conjunto de dados serão separados nos conjuntos de treinamento e de teste.


Vamos supor que desejamos criar um modelo para estimar o preço do veículo baseado nos seus outros atributos:

In [ ]:
# Dividindo x e y
# Foram excluídos as colunas que não seriam utilizadas
X = carros_corrigidos.drop(columns=['price'])
y = carros_corrigidos['price']

# Dividindo dados para treino e dados para teste
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 42)

Vamos ver o resultado que foi obtido para os `Dataframes` X_train, X_test, y_train e y_test:

In [ ]:
X_train.shape

(266, 11)

In [ ]:
X_test.shape

(114, 11)

Reparem que a proporção de 0.3 (30%) foi preservada entre o conjunto de teste e de treinamento:

In [ ]:
114/(266+114)

0.3

Notem que as 11 colunas de entrada (X) são iguais para os dois conjuntos de dados, o que muda são os índices das linhas.

In [ ]:
X_train.head()

,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,modelo_lounge,modelo_pop,modelo_sport,modelo_star
334,84,0,3500,94000,2,43.111359,12.38392,1,0,0,0
341,69,0,3804,87000,2,45.548222,9.18609,1,0,0,0
375,69,0,4474,55976,2,45.610050,9.24234,1,0,0,0
289,69,0,4474,55900,1,43.937881,12.71113,1,0,0,0
223,69,0,3197,139000,1,45.467960,9.18178,1,0,0,0


In [ ]:
X_test.head()

,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,modelo_lounge,modelo_pop,modelo_sport,modelo_star
266,69,0,4596,115000,1,45.783669,8.96724,0,0,1,0
261,69,0,3896,88296,1,43.332722,11.71933,0,1,0,0
265,69,0,3957,136000,1,41.152061,15.08309,1,0,0,0
39,69,0,762,36993,2,45.571220,9.15914,1,0,0,0
33,69,0,578,28000,1,41.903221,12.49565,1,0,0,0


Para os dados de saída, reparem que os índices são iguais para X_train e y_train e X_test e y_test, mantendo a correspondencia das linhas de entrada com as linhas de saída:

In [ ]:
y_train

334    5300
341    5399
375    5500
289    5100
223    4900
       ... 
71     9950
106    3650
270    4999
348    5400
102    3500
Name: price, Length: 266, dtype: int64

In [ ]:
y_test

266    4999
261    4990
265    4990
39     8980
33     8299
       ... 
137    4250
332    5300
351    5400
310    5200
167    4500
Name: price, Length: 114, dtype: int64

# Exercício)

Para o conjunto de dados Admission_Predict.csv faça a divisão do conjunto de dados em entrada e sáida para os conjuntos de treinamento e teste com 20% dos dados no conjunto de teste (saída dos dados: Coluna Chance). Alterne o valor do random_state. O que isso muda na divisão dos dados?

In [ ]:
# Seu código aquiteste

# Correção Exercício)

Para o conjunto de dados Admission_Predict.csv faça a divisão do conjunto de dados em entrada e sáida para os conjuntos de treinamento e teste com 20% dos dados no conjunto de teste (saída dos dados: coluna Chance). Alterne o valor do random_state. O que isso muda na divisão dos dados?

In [ ]:
# Recebendo os dados
alunos = pd.read_csv('/content/Admission_Predict.csv', sep = ',')
alunos.head()

,Serial_num.,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [ ]:
alunos.drop(columns=['Serial_num.'], inplace = True)
alunos.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [ ]:
# X = entrada; y = saída/label/rótulo
X = alunos.drop(columns=['Chance'])
y = alunos['Chance']
# Dividindo os dados em conjunto de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=10)

In [ ]:
X_train.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research
303,323,107,3,3.5,3.5,8.55,1
349,313,101,3,2.5,3.0,8.04,0
149,311,106,2,3.5,3.0,8.26,1
100,322,107,3,3.5,3.5,8.46,1
175,320,111,4,4.5,3.5,8.87,1


In [ ]:
X_test.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research
332,308,106,3,3.5,2.5,8.21,1
111,321,109,4,4.0,4.0,8.68,1
352,303,100,2,3.0,3.5,8.06,1
205,295,99,2,2.5,3.0,7.65,0
56,316,102,3,2.0,3.0,7.40,0
